In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/ratings_small.csv


In [64]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

In [3]:
m_data = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')

In [4]:
m_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
m_data['genres'] = m_data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [6]:
m_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [7]:
vote_counts = m_data[m_data['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = m_data[m_data['vote_average'].notnull()]['vote_count'].astype('int')
C = vote_averages.mean()

In [8]:
m = vote_counts.quantile(0.95)

In [9]:
m_data['year'] = pd.to_datetime(m_data['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan )

In [10]:
qualified = m_data[(m_data['vote_count'] > m) & (m_data['vote_count'].notnull()) & (m_data['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]

In [11]:
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')

In [12]:
# def weighted_rating(x):
#     v = x['vote_count']
#     r = x['vote_average']
#     return (v/(v+m) * r) + (m/(m+v) * C)


def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [13]:
qualified

,title,year,vote_count,vote_average,popularity,genres,wr
0,Toy Story,1995,5415,7,21.9469,"[Animation, Comedy, Family]",14.635056
1,Jumanji,1995,2413,6,17.0155,"[Adventure, Fantasy, Family]",21.838231
5,Heat,1995,1886,7,17.9249,"[Action, Crime, Drama, Thriller]",26.248899
9,GoldenEye,1995,1194,6,14.686,"[Adventure, Action, Thriller]",33.697448
15,Casino,1995,1343,7,10.1374,"[Drama, Crime]",32.130807
...,...,...,...,...,...,...,...
44624,What Happened to Monday,2017,598,7,60.581223,"[Science Fiction, Thriller]",50.272718
44632,Atomic Blonde,2017,748,6,14.455104,"[Action, Thriller]",44.148430
44678,Dunkirk,2017,2712,7,30.938854,"[Action, Drama, History, Thriller, War]",21.194992
44842,Transformers: The Last Knight,2017,1440,6,39.186819,"[Action, Science Fiction, Thriller, Adventure]",30.061603


In [14]:
qualified = qualified.sort_values('wr', ascending=False).head(200)
qualified.head(10)

,title,year,vote_count,vote_average,popularity,genres,wr
886,The Maltese Falcon,1941,435,7,5.78254,"[Mystery, Crime, Thriller]",58.389465
12350,The Girl Who Leapt Through Time,2006,435,7,12.4302,"[Fantasy, Animation, Drama, Science Fiction]",58.389465
1844,The French Connection,1971,435,7,6.69496,"[Action, Crime, Thriller]",58.389465
3189,City Lights,1931,444,8,10.8915,"[Comedy, Drama, Romance]",58.368388
16834,Jane Eyre,2011,438,7,7.08908,"[Drama, Romance]",58.212666
19518,Rust and Bone,2012,438,7,8.40005,"[Drama, Romance]",58.212666
32626,Suffragette,2015,440,7,8.09344,"[Drama, History]",58.095475
2755,Deliverance,1972,441,7,8.47668,"[Drama, Adventure, Thriller]",58.037080
5832,My Girl,1991,441,7,8.60574,"[Comedy, Drama, Family]",58.037080
33026,The Brand New Testament,2015,435,6,6.488451,"[Comedy, Fantasy]",57.888889


In [15]:
single = m_data.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
single.name = 'genres'
genres_m_data = m_data.drop('genres', axis=1).join(single)

In [16]:
genres_m_data

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genres
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Action
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Drama
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Thriller
45464,False,NaN,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917,NaN


In [17]:
def build_charts(genre, percentile=0.85):
    df = genres_m_data[genres_m_data['genres'] == genre]
    vote_count = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_average = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_average.mean()
    m = vote_count.quantile(percentile)
    
    qualified = df[(df['vote_count'] > m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count'] + m) * x['vote_average']) + (m / (m+x['vote_count']) * C), axis=1)
    
    qualified = qualified.sort_values('wr', ascending=False).head(200)
    
    return qualified

In [18]:
build_charts('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


In [19]:
#content-based
links_small = pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')

In [20]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [21]:
#Check EDA Notebook for how and why I got these indices.
m_data = m_data.drop([19730, 29503, 35587])# invalid data,
m_data['id'] = m_data['id'].astype('int')

In [22]:
small_md = m_data[m_data['id'].isin(links_small)]
small_md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40224,False,NaN,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",NaN,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,77000000.0,120.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,2016
40503,False,NaN,0,"[Documentary, Music]",http://www.thebeatlesliveproject.com/,391698,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,"The band stormed Europe in 1963, and, in 1964,...",...,0.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,False,7.6,92.0,2016
44821,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[Adventure, Fantasy, Animation, Action, Family]",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144.0,2000
44826,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[Adventure, Fantasy, Animation, Science Fictio...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,28023563.0,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82.0,2001


In [23]:
small_md['tagline'] = small_md['tagline'].fillna('')
small_md['description'] = small_md['overview'] + small_md['tagline']
small_md['description'] = small_md['description'].fillna('')
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')

In [24]:
tf_matrix = tf.fit_transform(small_md['description'])
tf_matrix.shape

(9099, 268124)

In [25]:
#Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. 
#Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.


cosine_sim = linear_kernel(tf_matrix, tf_matrix)

In [26]:
small_md = small_md.reset_index()
#small_md['title'] = small_md['title'].fillna('')
titles = small_md['title']
indices = pd.Series(small_md.index, index = small_md['title'])

In [27]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [28]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [29]:
#get_recommendations('intersteller').head(10)

In [30]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [31]:
credits = pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')

m_data['id'] = m_data['id'].astype('int')

In [32]:
m_data.shape

(45463, 25)

In [33]:
m_data = m_data.merge(credits, on='id')
m_data = m_data.merge(keywords, on='id')


In [34]:
smd = m_data[m_data['id'].isin(links_small)]
smd.shape

(9219, 28)

In [35]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9219 entries, 0 to 41669
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9219 non-null   object 
 1   belongs_to_collection  1688 non-null   object 
 2   budget                 9219 non-null   object 
 3   genres                 9219 non-null   object 
 4   homepage               2001 non-null   object 
 5   id                     9219 non-null   int64  
 6   imdb_id                9219 non-null   object 
 7   original_language      9219 non-null   object 
 8   original_title         9219 non-null   object 
 9   overview               9207 non-null   object 
 10  popularity             9219 non-null   object 
 11  poster_path            9216 non-null   object 
 12  production_companies   9219 non-null   object 
 13  production_countries   9219 non-null   object 
 14  release_date           9219 non-null   object 
 15  rev

In [36]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [37]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

smd['director'] = smd['crew'].apply(get_director)

In [38]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [39]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [40]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [41]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x])

In [42]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [43]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [44]:
s = s[s > 1]

In [45]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [46]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words


In [47]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [48]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [49]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [50]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9214
Rustom                                                9215
Mohenjo Daro                                          9216
Shin Godzilla                                         9217
The Beatles: Eight Days a Week - The Touring Years    9218
Length: 9219, dtype: int64

In [51]:
get_recommendations('The Dark Knight').head(10)

6218                 Batman Begins
8031         The Dark Knight Rises
7659    Batman: Under the Red Hood
1260                Batman & Robin
1134                Batman Returns
524                         Batman
2131                      Superman
132                 Batman Forever
8467                    Kick-Ass 2
8001              Batman: Year One
Name: title, dtype: object

In [52]:
def weighted_rating2(x, C, m):
    v = x['vote_count']
    R = x['vote_average']
    weight = (v/(v+m) * R) + (m/(m+v) * C)
    return weight


def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(lambda x: weighted_rating2(x, C, m), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified



In [53]:
#top25
improved_recommendations('The Dark Knight')#加了其他相似推荐了

,title,vote_count,vote_average,year,wr
8031,The Dark Knight Rises,9263,7,2012,6.904202
6218,Batman Begins,7511,7,2005,6.885231
7583,Kick-Ass,4747,7,2010,6.833082
524,Batman,2145,7,1989,6.708314
1134,Batman Returns,1706,6,1992,6.105404
8467,Kick-Ass 2,2275,6,2013,6.088795
8419,Man of Steel,6462,6,2013,6.041118
132,Batman Forever,1529,5,1995,5.578234
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.194327
1260,Batman & Robin,1447,4,1997,5.075329


In [54]:
improved_recommendations('The Prestige')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.947566
6981,The Dark Knight,12269,8,2008,7.940110
3381,Memento,4168,8,2000,7.834644
8031,The Dark Knight Rises,9263,7,2012,6.966120
6218,Batman Begins,7511,7,2005,6.958647
1023,Chinatown,939,7,1974,6.760449
1490,The X Files,494,6,1998,6.111993
7523,Edge of Darkness,496,6,2010,6.111752
2430,The Bone Collector,843,6,1999,6.081343
4145,Insomnia,1181,6,2002,6.064300


In [55]:
improved_recommendations('Pulp Fiction')

,title,vote_count,vote_average,year,wr
8905,The Hateful Eight,4405,7,2015,6.934371
5200,Kill Bill: Vol. 2,4061,7,2004,6.929206
3771,Training Day,1665,7,2001,6.843294
1381,Jackie Brown,1580,7,1997,6.836244
8110,The Raid,1076,7,2011,6.776671
1532,The French Connection,435,7,1971,6.584363
3249,Traffic,573,6,2000,6.000000
3012,Shaft,316,5,2000,5.494723
4764,S.W.A.T.,780,5,2003,5.284010
7104,The Spirit,323,4,2008,4.978495


In [56]:
#collaborative filter
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')

In [57]:
reader = Reader()

In [58]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [71]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25)

In [72]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9002  0.9027  0.8934  0.9011  0.8963  0.8987  0.0034  
MAE (testset)     0.6910  0.6913  0.6896  0.6935  0.6920  0.6915  0.0013  
Fit time          5.58    5.52    5.45    5.38    5.36    5.46    0.08    
Test time         0.18    0.37    0.17    0.17    0.34    0.25    0.09    


{'test_rmse': array([0.90023265, 0.90266613, 0.89342099, 0.90109986, 0.89628771]),
 'test_mae': array([0.69095553, 0.69131622, 0.68961908, 0.69349376, 0.69203976]),
 'fit_time': (5.575703144073486,
  5.515147686004639,
  5.4467644691467285,
  5.381133556365967,
  5.359747409820557),
 'test_time': (0.18146133422851562,
  0.37291502952575684,
  0.1715378761291504,
  0.1728837490081787,
  0.34299349784851074)}

In [73]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [74]:
predictions = svd.test(testset)

In [76]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.6425


0.6424625482175392

In [77]:
from surprise.model_selection import KFold


kf = KFold(n_splits=5)

algo = SVD()

for trainset, testset in kf.split(data):

    # 训练并测试算法
    algo.fit(trainset)
    predictions = algo.test(testset)

    # 计算并打印 RMSE（均方根误差，Root Mean Squared Error）
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8927
RMSE: 0.8881
RMSE: 0.9009
RMSE: 0.9002
RMSE: 0.9021


In [78]:
svd.predict(1, 302, 3)


Prediction(uid=1, iid=302, r_ui=3, est=2.762003225094685, details={'was_impossible': False})

In [79]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [81]:
id_map = pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')[['movieId', 'tmdbId']]#只取这两个
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [82]:
id_map

,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0
...,...,...
The Last Brickmaker in America,161944,159550.0
Rustom,162542,392572.0
Mohenjo Daro,162672,402672.0


In [83]:
indices_map = id_map.set_index('id')

In [84]:
indices_map

,movieId
id,
862.0,1
8844.0,2
15602.0,3
31357.0,4
11862.0,5
...,...
159550.0,161944
392572.0,162542
402672.0,162672


In [85]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [88]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
974,Aliens,3282.0,7.7,1986,679,3.286577
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.904757
8488,Gravity,5879.0,7.3,2013,49047,2.891250
987,Alien,4564.0,7.9,1979,348,2.890782
7289,Star Trek,4574.0,7.4,2009,13475,2.886895
7068,Stargate: Continuum,151.0,6.8,2008,12914,2.855216
6316,Star Wreck: In the Pirkinning,27.0,6.6,2005,15493,2.847315
2452,Spaceballs,924.0,6.7,1987,957,2.839651
3819,Mission to Mir,2.0,3.0,1997,15560,2.818382
4404,Treasure Planet,980.0,7.2,2002,9016,2.751864


In [89]:
hybrid(1, 'Alien')

,title,vote_count,vote_average,year,id,est
974,Aliens,3282.0,7.7,1986,679,3.286577
8712,Guardians of the Galaxy,10014.0,7.9,2014,118340,3.119876
8983,The Martian,7442.0,7.6,2015,286217,3.034592
8488,Gravity,5879.0,7.3,2013,49047,2.891250
6316,Star Wreck: In the Pirkinning,27.0,6.6,2005,15493,2.847315
2452,Spaceballs,924.0,6.7,1987,957,2.839651
3791,SpaceCamp,60.0,5.8,1986,13766,2.731498
2920,Moonraker,551.0,5.9,1979,698,2.716412
5292,Babylon 5: A Call to Arms,36.0,6.9,1999,10916,2.681371
4153,Silent Running,179.0,6.2,1972,811,2.654610


In [ ]:
#content-based + collaborative filtering